In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import keras
from datetime import datetime
from pynput.mouse import Button,Controller
mouse = Controller()

def get_extended_image(img, x, y, w, h, k=0.1):
    if x - k*w > 0:
        start_x = int(x - k*w)
    else:
        start_x = x
    if y - k*h > 0:
        start_y = int(y - k*h)
    else:
        start_y = y

    end_x = int(x + (1 + k)*w)
    end_y = int(y + (1 + k)*h)

    face_image = img[start_y:end_y,
                     start_x:end_x]
    face_image = tf.image.resize(face_image, [224, 224])
    face_image = np.expand_dims(face_image, axis=0)
    return face_image

video_capture = cv2.VideoCapture(0, cv2.CAP_DSHOW)

face_classifier = keras.models.load_model('ModelPath')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
class_names = ['Gozler Kapali', 'Sol Goz Kirpiliyor','Gozler Acik','Sag Goz Kirpiliyor']

now = datetime.now()
gozlerkapali=False
while True:
    ret, frame = video_capture.read()
    if not ret:
        print("\n\n\nCan't receive frame (stream end?). Exiting ...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=5,
        minSize=(100, 100),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    for (x, y, w, h) in faces:
        face_image = get_extended_image(frame, x, y, w, h, 0.5)
        result = face_classifier.predict(face_image)
        prediction = class_names[np.array(
            result[0]).argmax(axis=0)]  
        confidence = np.array(result[0]).max(axis=0)  
        if prediction == 'Gozler Kapali':
            if gozlerkapali==False:
                now=datetime.now()
                gozlerkapali=True
                birkeretikla=False
            else:
                now2=datetime.now()
                if (now2-now).microseconds > 600000:
                    color = (0,0,255)  #red
                    if birkeretikla==False:
                        mouse.click(Button.left,1)
                        birkeretikla=True
        elif prediction == 'Sol Goz Kirpiliyor':
            mouse.scroll(0,-1)
            color = (255,0,0) 
            gozlerkapali=False
        elif prediction == 'Gozler Acik':
            color = (255,255,255)
            gozlerkapali=False
        elif prediction == 'Sag Goz Kirpiliyor':
            color = (0,255,0)
            mouse.scroll(0,1)
            gozlerkapali=False
        cv2.rectangle(frame,
                      (x, y),  
                      (x+w, y+h),
                      color,
                      2) 
        cv2.putText(frame,
                    "{:6} - {:.2f}%".format(prediction, confidence*100),
                    (x, y),
                    cv2.FONT_HERSHEY_PLAIN,
                    2,  
                    color,
                    2) 

    cv2.imshow("Face detector - to quit press ESC", frame)
    key = cv2.waitKey(1)
    if key % 256 == 27:  
        break


video_capture.release()
cv2.destroyAllWindows()
print("\n\n\nStreaming ended")
